In [1]:
import socket
import shutil
from gcsfs import GCSFileSystem
import os
from dask.distributed import Client, progress
client = Client()
client

Client Scheduler: tcp://scheduler:8786 Dashboard: http://scheduler:8787/status,Cluster Workers: 3 Cores: 24 Memory: 49.49 GB


In [2]:
DASK_DATA_WORKERS = ["worker-1:3001", "worker-2:3002"]

In [3]:
#from gcsfs import GCSFileSystem
#import os
#gcs = GCSFileSystem(token='anon')
#parquet_files = gcs.ls('anaconda-public-data/nyc-taxi/nyc.parquet')
#parquet_files

In [4]:
def download_parquet_files(local_parquet='/opt/data/nyc.parquet', clean=True):
    """
    Download parquet files
    """
    gcs = GCSFileSystem(token='anon')

    parquet_files = gcs.ls('anaconda-public-data/nyc-taxi/nyc.parquet')
    parquet_files

    # clean directory
    if os.path.exists(local_parquet) and clean:
        shutil.rmtree(local_parquet)
    # create directory
    if not os.path.exists(local_parquet):
        os.makedirs(local_parquet)
    # download files
    for f in parquet_files[:5]:
        parquet_files = gcs.download(f, f'/{local_parquet}/{os.path.basename(f)}')

In [5]:
def run_parquet_files_downloader(client, workers):
    """
    Run parquet files downloader
    """
    args = ()
    result = client.run(download_parquet_files, *args, workers=workers)

In [ ]:
run_parquet_files_downloader(client, DASK_DATA_WORKERS)